# Tests

In [ ]:
import pandas as pd
import numpy as np
import os

!pip install control
from tools import *

In [ ]:
# FILE_PATH = r'C:\Users\guisa\Google Drive\01 - Iniciação Científica\02 - Datasets\csv_files\EN2_STAR_CHR_0101086161_20070516T060226_20071005T074409.csv'
FILE_PATH = '/content/drive/MyDrive/01 - Iniciação Científica/02 - Datasets/csv_files/EN2_STAR_CHR_0101086161_20070516T060226_20071005T074409.csv'

Reading one time series

In [ ]:
data = pd.read_csv(FILE_PATH)

data.head()

,DATE,WHITEFLUX
0,2007-05-16 18:10:55.071642,112626.77
1,2007-05-16 18:19:27.113766,112605.61
2,2007-05-16 18:27:59.155929,112771.50
3,2007-05-16 18:36:31.198092,113113.60
4,2007-05-16 18:45:03.240256,112621.79


Taking only the flux and "saving" it as `pd.Series`

In [ ]:
flux = data.WHITEFLUX
flux

0        32847.277
1        32849.570
2        32850.133
3        32881.668
4        32897.824
           ...    
24451    33611.984
24452    33498.582
24453    33537.754
24454    33552.040
24455    33650.860
Name: WHITEFLUX, Length: 24456, dtype: float64

Creating a `pd.DataFrame` and appending the data from `flux` to the first column and the first row.

Later, we are going to do the same for lightcurves

In [ ]:
# to SkTime
DF = pd.DataFrame()

DF = DF.append([[flux]], ignore_index=True)
DF.columns = ['time_series']

DF.head()

###############################################################################

# to SKLEARN
# DF = pd.DataFrame()

# DF = DF.append(flux, ignore_index=True)

# DF.head()

,time_series
0,0 32847.277 1 32849.570 2 ...


In [ ]:
DF.iloc[0][0]

0        32847.277
1        32849.570
2        32850.133
3        32881.668
4        32897.824
           ...    
24451    33611.984
24452    33498.582
24453    33537.754
24454    33552.040
24455    33650.860
Name: WHITEFLUX, Length: 24456, dtype: float64

# SkTime

In [ ]:
import pandas as pd
import numpy as np
import os

!pip install control
from tools import *

## Preprocessing data 

###  Creating matrix of features (CoRoT targets with confirmed exoplanets)


In [3]:
# DATA_DIR = 'C:/Users/guisa/Google Drive/01 - Iniciação Científica/02 - Datasets/csv_files'
# DATA_DIR = '/content/drive/MyDrive/01 - Iniciação Científica/02 - Datasets/csv_files'
DATA_DIR = '/content/drive/MyDrive/01 - Iniciação Científica/02 - Datasets/resampled_files'

In [4]:
X = pd.DataFrame()

for root_dir_path, sub_dirs, files in os.walk(DATA_DIR):
    for j in range(0, len(files)):
        if files[j] != ('desktop.ini' and 'csv_files.rar'):
            # File path
            path = root_dir_path + "/" + files[j] 
            
            # Reading data
            # print(path)
            data = pd.read_csv(path)
            flux = data.WHITEFLUX
            
            # Add timeseries to pd.DataFrame
            X = X.append([[flux]], ignore_index=True)

In [5]:
X.columns = ['time_series']
X.head()

,time_series
0,0 141157.216020 1 141242.434636 ...
1,0 26051.806326 1 26113.304520 2 ...
2,0 1.298393e+06 1 1.299550e+06 2 ...
3,0 112521.329834 1 112758.045853 ...
4,0 40643.684746 1 40245.967717 2 ...


In [6]:
X.iloc[0][0]

0        141157.216020
1        141242.434636
2        141132.564812
3        141373.143346
4        141213.262888
             ...      
15045    141147.423729
15046    141100.943892
15047    141378.202818
15048    141228.656766
15049    141309.960163
Name: WHITEFLUX, Length: 15050, dtype: float64

In [7]:
X.shape

(33, 1)

### Labeling matrix of features

- 0: confirmed_exoplanets
- 1: eclipsing_binaries
- 2: none

In [8]:
labels = np.zeros(X.size, dtype='int')
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [9]:
y = pd.Series(labels)
y.head()

0    0
1    0
2    0
3    0
4    0
dtype: int64

In [10]:
y.shape

(33,)

### Creating dataset, X and y

In [11]:
# Creating pd.DataFrame with X data, and setted columns
df = pd.DataFrame(X, columns=['time_series', 'label'])

# Adding labels
df.label = y

df.head()

,time_series,label
0,0 141157.216020 1 141242.434636 ...,0
1,0 26051.806326 1 26113.304520 2 ...,0
2,0 1.298393e+06 1 1.299550e+06 2 ...,0
3,0 112521.329834 1 112758.045853 ...,0
4,0 40643.684746 1 40245.967717 2 ...,0


How many Labels we got ?

In [12]:
labels, counts = np.unique(y, return_counts=True)
print('Labels =', labels, '\nCounts =', counts)

Labels = [0] 
Counts = [33]


## Machine Learning - SkTime

https://github.com/alan-turing-institute/sktime/tree/v0.4.3

https://github.com/alan-turing-institute/sktime/blob/main/sktime/classification/compose/__init__.py

### **Preliminaries**

In [ ]:
# !pip install sktime[all_extras]

### **Splitting the dataset into the Training set and Test set**

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(24, 1) (24,) (9, 1) (9,)


### **Time Series Classification**


https://towardsdatascience.com/sktime-a-unified-python-library-for-time-series-machine-learning-3c103c139a55

In [14]:
from sktime.classification.all import TimeSeriesForestClassifier

classifier = TimeSeriesForestClassifier()
classifier.fit(X_train, y_train)

TimeSeriesForestClassifier()

In [15]:
from sklearn.metrics import accuracy_score

y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

1.0

### **Feature extraction**

In [28]:
import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.ar_model.AR', FutureWarning)

In [29]:
from sktime.transformations.panel.tsfresh import TSFreshFeatureExtractor

transformer = TSFreshFeatureExtractor(default_fc_parameters="minimal")

extracted_features = transformer.fit_transform(X_train)
extracted_features.head()


Feature Extraction: 100%|██████████| 5/5 [00:00<00:00, 18.29it/s]


,time_series__sum_values,time_series__median,time_series__mean,time_series__length,time_series__standard_deviation,time_series__variance,time_series__root_mean_square,time_series__maximum,time_series__minimum
0,2.124862e+09,141218.330072,141186.836848,15050.0,282.683706,79910.077814,141187.119841,142021.360654,138999.945791
4,6.136100e+08,40704.273331,40771.427763,15050.0,560.265206,313897.101078,40775.277055,44921.695568,39239.685767
16,6.258849e+08,41653.547836,41587.034197,15050.0,364.389989,132780.064141,41588.630578,42472.076073,40477.653465
5,4.512183e+09,299686.539820,299812.826420,15050.0,613.298072,376134.525086,299813.453701,302311.608084,297465.565232
13,5.857504e+08,38822.561668,38920.292034,15050.0,553.820584,306717.239608,38924.232160,40531.501716,37855.665849


In [25]:
# If the result is 1, it means that the entire dataset has de same lenght

extracted_features.time_series__length.nunique()

1

### **Time Series Classification with Feature Extraction**

In [32]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

classifier = make_pipeline(
    TSFreshFeatureExtractor(show_warnings=False), RandomForestClassifier()
)
classifier.fit(X_train, y_train)
classifier.score(X_test, y_test)


Feature Extraction: 100%|██████████| 5/5 [07:48<00:00, 93.60s/it]

Feature Extraction: 100%|██████████| 5/5 [02:50<00:00, 34.17s/it]


1.0

# SkLearn

In [ ]:
import pandas as pd
import numpy as np
import os

!pip install control
from tools import *

## Preprocessing data

###  Creating matrix of features (CoRoT targets with confirmed exoplanets)


In [55]:
# DATA_DIR = 'C:/Users/guisa/Google Drive/01 - Iniciação Científica/02 - Datasets/csv_files'
# DATA_DIR = '/content/drive/MyDrive/01 - Iniciação Científica/02 - Datasets/csv_files'
DATA_DIR = '/content/drive/MyDrive/01 - Iniciação Científica/02 - Datasets/resampled_files'

In [56]:
X = pd.DataFrame()

for root_dir_path, sub_dirs, files in os.walk(DATA_DIR):
    for j in range(0, len(files)):
        if files[j] != ('desktop.ini' and 'csv_files.rar'):
            # File path
            path = root_dir_path + "/" + files[j] 
            
            # Reading data
            # print(path)
            data = pd.read_csv(path)
            flux = data.WHITEFLUX
            
            # Add timeseries to pd.DataFrame
            X = X.append(flux, ignore_index=True)

In [57]:
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,15010,15011,15012,15013,15014,15015,15016,15017,15018,15019,15020,15021,15022,15023,15024,15025,15026,15027,15028,15029,15030,15031,15032,15033,15034,15035,15036,15037,15038,15039,15040,15041,15042,15043,15044,15045,15046,15047,15048,15049
0,1.411572e+05,1.412424e+05,1.411326e+05,1.413731e+05,1.412133e+05,1.413579e+05,1.411959e+05,1.412497e+05,1.413829e+05,1.414096e+05,1.412703e+05,1.412327e+05,1.411492e+05,1.412873e+05,1.412144e+05,1.411442e+05,1.412001e+05,1.412225e+05,1.413235e+05,1.412076e+05,1.411856e+05,1.411647e+05,1.412128e+05,1.413366e+05,1.412635e+05,1.413088e+05,1.413307e+05,1.411438e+05,1.412988e+05,1.411502e+05,1.411364e+05,1.414083e+05,1.410876e+05,1.411729e+05,1.412924e+05,1.413673e+05,1.412330e+05,1.411206e+05,1.413310e+05,1.411721e+05,...,1.411859e+05,1.411025e+05,1.410474e+05,1.411354e+05,1.411319e+05,1.412317e+05,1.411159e+05,1.412205e+05,1.411621e+05,1.411025e+05,1.412063e+05,1.411235e+05,1.411927e+05,1.412480e+05,1.412074e+05,1.411396e+05,1.411576e+05,1.410457e+05,1.411399e+05,1.408885e+05,1.409958e+05,1.411260e+05,1.411951e+05,1.411575e+05,1.411057e+05,1.411124e+05,1.412933e+05,1.411076e+05,1.410541e+05,1.411741e+05,1.409121e+05,1.409989e+05,1.412270e+05,1.411838e+05,1.413012e+05,1.411474e+05,1.411009e+05,1.413782e+05,1.412287e+05,1.413100e+05
1,2.605181e+04,2.611330e+04,2.601663e+04,2.614152e+04,2.587125e+04,2.587146e+04,2.602901e+04,2.604010e+04,2.611140e+04,2.607349e+04,2.612061e+04,2.601961e+04,2.608381e+04,2.615512e+04,2.605209e+04,2.615393e+04,2.595893e+04,2.613356e+04,2.608572e+04,2.604287e+04,2.609076e+04,2.603981e+04,2.604673e+04,2.605018e+04,2.605711e+04,2.603637e+04,2.602297e+04,2.610288e+04,2.599291e+04,2.599706e+04,2.596696e+04,2.610334e+04,2.616852e+04,2.615221e+04,2.600709e+04,2.604340e+04,2.602915e+04,2.623798e+04,2.596439e+04,2.610146e+04,...,2.621469e+04,2.632878e+04,2.624308e+04,2.622704e+04,2.618278e+04,2.624397e+04,2.632099e+04,2.627644e+04,2.625360e+04,2.632872e+04,2.625611e+04,2.633979e+04,2.629440e+04,2.627620e+04,2.635345e+04,2.628232e+04,2.634446e+04,2.636739e+04,2.625771e+04,2.648313e+04,2.638753e+04,2.626968e+04,2.623826e+04,2.630487e+04,2.624348e+04,2.638560e+04,2.620561e+04,2.630678e+04,2.627786e+04,2.616603e+04,2.631367e+04,2.620188e+04,2.618542e+04,2.624512e+04,2.625785e+04,2.642315e+04,2.621238e+04,2.636027e+04,2.629231e+04,2.618336e+04
2,1.298393e+06,1.299550e+06,1.299725e+06,1.299612e+06,1.299747e+06,1.299215e+06,1.299576e+06,1.299769e+06,1.299262e+06,1.299409e+06,1.299280e+06,1.299889e+06,1.299150e+06,1.299826e+06,1.298902e+06,1.299552e+06,1.299346e+06,1.298708e+06,1.299628e+06,1.299107e+06,1.299239e+06,1.299363e+06,1.299605e+06,1.299160e+06,1.299955e+06,1.299210e+06,1.299477e+06,1.299130e+06,1.299318e+06,1.298997e+06,1.299127e+06,1.299335e+06,1.299339e+06,1.299389e+06,1.299585e+06,1.299507e+06,1.298837e+06,1.299754e+06,1.298997e+06,1.300436e+06,...,1.295584e+06,1.296259e+06,1.295880e+06,1.296397e+06,1.295613e+06,1.295237e+06,1.295789e+06,1.295417e+06,1.295453e+06,1.295508e+06,1.295937e+06,1.294957e+06,1.295125e+06,1.294599e+06,1.294709e+06,1.295073e+06,1.295429e+06,1.295154e+06,1.295264e+06,1.295769e+06,1.295695e+06,1.295337e+06,1.295557e+06,1.295314e+06,1.295710e+06,1.295153e+06,1.295031e+06,1.295029e+06,1.295460e+06,1.295186e+06,1.294849e+06,1.295283e+06,1.294897e+06,1.294750e+06,1.294881e+06,1.294939e+06,1.295167e+06,1.295158e+06,1.295069e+06,1.294454e+06
3,1.125213e+05,1.127580e+05,1.129430e+05,1.125623e+05,1.127893e+05,1.125752e+05,1.127852e+05,1.126351e+05,1.126462e+05,1.126747e+05,1.128206e+05,1.126230e+05,1.127497e+05,1.127325e+05,1.127567e+05,1.127885e+05,1.126766e+05,1.127609e+05,1.125398e+05,1.127966e+05,1.126471e+05,1.126480e+05,1.128501e+05,1.128040e+05,1.127078e+05,1.128669e+05,1.126771e+05,1.127147e+05,1.127916e+05,1.126816e+05,1.127761e+05,1.126781e+05,1.127678e+05,1.127868e+05,1.125911e+05,1.127481e+05,1.127409e+05,1.126717e+05,1.126739e+0

In [58]:
X.shape

(33, 15050)

### Labeling matrix of features

- 0: confirmed_exoplanets
- 1: eclipsing_binaries
- 2: none

In [78]:
labels = np.zeros(X.size, dtype='int')
labels

array([0, 0, 0, ..., 0, 0, 0])

In [79]:
y = pd.Series(labels)
y.head()

0    0
1    0
2    0
3    0
4    0
dtype: int64

In [80]:
y.shape

(496650,)

### Creating dataset, X and y

In [81]:
# Creating pd.DataFrame with X data, and setted columns
df = pd.DataFrame(X)

# Adding labels
df['label'] = y

df.sample(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,15011,15012,15013,15014,15015,15016,15017,15018,15019,15020,15021,15022,15023,15024,15025,15026,15027,15028,15029,15030,15031,15032,15033,15034,15035,15036,15037,15038,15039,15040,15041,15042,15043,15044,15045,15046,15047,15048,15049,label
7,30819.826000,30766.330239,30751.556425,30749.078411,30735.472031,30705.364719,30673.455704,30662.058557,30682.811774,30726.006400,30766.044428,30779.528462,30762.480430,30733.670298,30721.967523,30747.701120,30810.995926,30892.582773,30963.134137,30994.637790,30971.285524,30899.489245,30812.158646,30757.956592,30773.076489,30850.996851,30937.516699,30965.531315,30909.779524,30815.873666,30770.754493,30830.580119,30965.570660,31074.862790,31066.062397,30933.718685,30766.639144,30675.022829,30700.963010,30791.397016,...,31257.034885,31264.336563,31300.065245,31342.684879,31375.525377,31386.046775,31362.258516,31301.359103,31225.968013,31185.069625,31225.200043,31348.360808,31494.844160,31574.995403,31532.203536,31387.986425,31230.251847,31151.297131,31183.599756,31282.844881,31368.501950,31386.865644,31347.153080,31306.651124,31321.815477,31405.234907,31518.720306,31602.739344,31617.101315,31564.063024,31481.233700,31413.127641,31382.205800,31377.323280,31364.368751,31310.075393,31203.441067,31062.404104,30923.186993,0
0,141157.216020,141242.434636,141132.564812,141373.143346,141213.262888,141357.927056,141195.854576,141249.723060,141382.882136,141409.646127,141270.347310,141232.742194,141149.203020,141287.347367,141214.443625,141144.196655,141200.096238,141222.533249,141323.471805,141207.648435,141185.552578,141164.668742,141212.764561,141336.627107,141263.521595,141308.810296,141330.700205,141143.809870,141298.757841,141150.205593,141136.397407,141408.260213,141087.606264,141172.925450,141292.442324,141367.329433,141232.994283,141120.579153,141330.990802,141172.077723,...,141102.476944,141047.400969,141135.385006,141131.922378,141231.656466,141115.855797,141220.542768,141162.097924,141102.469348,141206.299004,141123.514021,141192.723871,141248.047431,141207.440681,141139.602971,141157.554097,141045.650680,141139.907409,140888.479585,140995.832130,141126.000693,141195.104186,141157.523028,141105.662748,141112.441533,141293.285098,141107.636577,141054.134890,141174.120525,140912.054405,140998.875288,141227.024526,141183.770462,141301.178908,141147.423729,141100.943892,141378.202818,141228.656766,141309.960163,0
28,62789.448650,63084.529078,62888.248116,62879.160690,62902.299203,62856.785176,62838.438216,62890.327459,62978.468824,62915.586396,63030.280126,62972.033937,63002.387872,62917.744686,62882.950698,62963.445695,63047.080991,62936.457158,62918.843671,62903.944131,62860.251827,62981.319394,62955.393306,62950.430855,62790.031253,62864.886726,62973.441523,62832.014189,62840.955762,62894.206974,62961.589149,63016.263609,62849.370593,62860.779191,62823.774972,62836.445243,62939.162851,63026.291359,63070.556424,62920.886694,...,62681.452641,62818.463320,62658.489891,62751.509989,62673.658088,62645.568598,62681.461059,62508.688815,62410.438476,62357.747106,62424.788206,62287.583036,62463.003606,62461.858904,62661.955794,62695.748268,62763.846757,62617.561068,62713.372410,62667.105538,62616.660149,62616.715432,62717.656437,62704.556538,62715.951555,62749.903309,62744.565477,62690.301100,62640.328884,62562.900647,62681.503019,62595.074849,62785.115156,62726.137782,62662.004908,62639.195962,62595.452252,62665.525918,62690.144817,0
16,41432.330660,41693.673194,41615.060995,41733.206360,41407.073111,41575.178851,41481.167264,41461.696150,41574.132506,41545.980086,41589.135266,41631.806337,41678.861443,41666.674866,41573.258211,41595.650027,41704.488771,41548.087189,41514.108672,41592.132381,41587.728893,41642.447618,41577.163200,41561.930921,41516.422593,41608.456849,41626.788574,41504.738696,41597.093006,41628.753414,41551.683274,41666.509823,41582.059959,41581.388399,41583.151091,41

How many Labels we got ?

In [82]:
labels, counts = np.unique(y, return_counts=True)
print('Labels =', labels, '\nCounts =', counts)

Labels = [0] 
Counts = [496650]


## Machine Learning

### **Preliminaries**

### **Splitting the dataset into the Training set and Test set**

In [83]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [84]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(24, 15050) (24,) (9, 15050) (9,)


### **Time Series Classification**

In [91]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)

# from sklearn import svm
# classifier = svm.SVC()

classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [86]:
y_pred = classifier.predict(X_test)

In [87]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[9]]


1.0